In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar los datos procesados
data = pd.read_csv('datos_aleatorios_resultado.csv', sep=',')

# Asegurar tipos adecuados
data['Genero'] = pd.to_numeric(data['Genero'], errors='coerce')
data['Edad'] = pd.to_numeric(data['Edad'], errors='coerce')
data['Estadio ERC'] = pd.to_numeric(data['Estadio ERC'], errors='coerce')

# Agrupación por Estadio ERC con estadísticas de Edad y proporción de Género
resumen = data.groupby('Estadio ERC').agg(
    Edad_Promedio=('Edad', 'mean'),
    Proporcion_Masculino=('Genero', lambda x: (x == 1).mean()),
    Total_Pacientes=('Genero', 'count')
).reset_index()

# Mostrar resumen
print(resumen)

import plotly.express as px

# Suponiendo que 'resumen' tiene columnas: 'Estadio ERC' y 'Edad_Promedio'
fig = px.bar(resumen,
             x='Estadio ERC',
             y='Edad_Promedio',
             color='Edad_Promedio',
             text='Edad_Promedio',
             color_continuous_scale='Bluered_r',
             title='Edad Promedio por Estadio ERC')

fig.update_traces(texttemplate='%{text:.1f} años', textposition='outside')
fig.update_layout(yaxis_title='Edad Promedio (años)',
                  xaxis_title='Estadio ERC',
                  coloraxis_showscale=False,
                  uniformtext_minsize=8,
                  uniformtext_mode='hide')

plt.savefig("mi_app/static/edad_promedio_por_estadio.jpg")

# Crear df_melted para proporciones de género por estadio
df_melted = pd.DataFrame({
    'Estadio ERC': resumen['Estadio ERC'].tolist() * 2,
    'Proporción': pd.concat([
        resumen['Proporcion_Masculino'],
        1 - resumen['Proporcion_Masculino']
    ], ignore_index=True),
    'Género': ['Masculino'] * len(resumen) + ['Femenino'] * len(resumen)
})

import plotly.express as px

# Añadir columna Proporcion_Femenino
resumen['Proporcion_Femenino'] = 1 - resumen['Proporcion_Masculino']

df_melted = resumen.melt(id_vars='Estadio ERC', 
                         value_vars=['Proporcion_Masculino', 'Proporcion_Femenino'],
                         var_name='Género', 
                         value_name='Proporción')

df_melted['Género'] = df_melted['Género'].map({
    'Proporcion_Masculino': 'Masculino',
    'Proporcion_Femenino': 'Femenino'
})

# Crear gráfico interactivo
fig = px.bar(df_melted, 
             x='Estadio ERC', 
             y='Proporción', 
             color='Género', 
             barmode='group',
             title='Comparativa Interactiva: Proporción Masculina vs Femenina por Estadio ERC',
             text_auto=True,
             color_discrete_sequence=['#3366CC', '#FF6666'])

fig.update_layout(yaxis=dict(range=[0, 1]))
plt.savefig("mi_app/static/proporcion_masculino_vs_femenino.jpg")

   Estadio ERC  Edad_Promedio  Proporcion_Masculino  Total_Pacientes
0            1      51.200000              0.420000               50
1            2      53.800000              0.500000               50
2            3      57.020000              0.600000               50
3            4      62.160000              0.680000               50
4            5      68.806452              0.774194               31


FileNotFoundError: [Errno 2] No such file or directory: 'mi_app/static/edad_promedio_por_estadio.jpg'

<Figure size 640x480 with 0 Axes>